In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LinearRegression
import time

Задача: обучить программу определять по имющимся данным вид раковой опухили (доброкачественная или злокачественная). Это задача бинарной классификации.
Дата сет: https://www.kaggle.com/uciml/breast-cancer-wisconsin-data/data

In [8]:
file_path = 'drive/My Drive/Data/data.csv'

data = pd.read_csv(file_path)

data.shape

(569, 33)

In [9]:
pd.set_option('display.max_columns', 32)
#data.describe()
data.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,...,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,...,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,...,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,...,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,...,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,...,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [0]:
data.drop(data.columns[32], inplace=True, axis=1)

In [11]:
categorical_columns = [c for c in data.columns if data[c].dtype.name == 'object']
numerical_columns   = [c for c in data.columns if data[c].dtype.name != 'object']
data[categorical_columns].describe()

,diagnosis
count,569
unique,2
top,B
freq,357


In [12]:
data_describe = data.describe(include=[object])
for c in categorical_columns:
    top = data_describe[c]['top']
    top_items = data[c] == top
    data.loc[top_items, c] = int(0)
    data.loc[np.logical_not(top_items), c] = int(1)
data[categorical_columns].describe()

,diagnosis
count,569
unique,2
top,0
freq,357


In [0]:
data_numerical = data[numerical_columns]

scale_features_std = StandardScaler() 
data_numerical = scale_features_std.fit_transform(data_numerical) 

scale_features_mm = MinMaxScaler() 
data_numerical = scale_features_mm.fit_transform(data_numerical)
data_numerical = pd.DataFrame(data_numerical)

In [14]:
y = data['diagnosis']
data.drop('diagnosis', inplace=True, axis=1)
y

0      1
1      1
2      1
3      1
4      1
      ..
564    1
565    1
566    1
567    1
568    0
Name: diagnosis, Length: 569, dtype: object

In [0]:
X_train, X_test, y_train, y_test = train_test_split(data, y.astype('int'), test_size = 0.3, random_state = 11)

In [16]:
lir = LinearRegression()
start_time = time.time()
lir.fit(X_train, y_train)

err_test  = np.mean(y_test  != lir.predict(X_test))
print('Линейная регрессия: ',err_test)
print('Время на обучение: ', time.time() - start_time, ' секунд')

Линейная регрессия:  1.0
Время на обучение:  0.04529142379760742  секунд


Очень большая ошибка. Логистическая регрессия плохо спровляется с поставленной задачей.

In [17]:
lr = LogisticRegression()
start_time = time.time()
lr.fit(X_train, y_train)

err_test  = np.mean(y_test  != lr.predict(X_test))
print('Логистическая регрессия: ',err_test)
print('Время на обучение: ', time.time() - start_time, ' секунд')

Логистическая регрессия:  0.3567251461988304
Время на обучение:  0.015670299530029297  секунд


У Дерева Решений хорошая ошибка при заметно более низком времени, чем у алгоритмов с минимальной ошибкой. Этот алгоритм хорошо справился с данной задачей.

In [18]:
dtc = DecisionTreeClassifier()
start_time = time.time()
dtc.fit(X_train, y_train)

err_test  = np.mean(y_test  != dtc.predict(X_test))
print('Дерево решений: ',err_test)
print('Время на обучение: ', time.time() - start_time, ' секунд')

Дерево решений:  0.07017543859649122
Время на обучение:  0.015016794204711914  секунд


У Наивного Баеса крайне высокая ошибка. Пока алгоритм полностью бесполезен для решения задачи. Возможно ситуация улучшиться, если подобрать правильные параметры.

In [19]:
mnb = MultinomialNB()
start_time = time.time()
mnb.fit(X_train, y_train)

err_test  = np.mean(y_test  != mnb.predict(X_test))
print('Наивный Байес: ',err_test)
print('Время на обучение: ', time.time() - start_time, ' секунд')

Наивный Байес:  0.6900584795321637
Время на обучение:  0.008867263793945312  секунд


У Метода ближайших сосесдей большая ошибка, при этом время хуже, чем, к примеру, у Деревьев решений.

In [26]:
knn = KNeighborsClassifier()
start_time = time.time()
knn.fit(X_train, y_train)

err_test  = np.mean(y_test  != knn.predict(X_test))
print('Метод ближайших соседей: ',err_test)
print('Время на обучение: ', time.time() - start_time, ' секунд')

Математическое ожидание ошибки:  0.22807017543859648
Время на обучение:  0.015660762786865234  секунд


У Метода опорных векторов большая ошибка, при этом время хуже, чем, к примеру, у Деревьев решений.

In [21]:
svc = svm.SVC()
start_time = time.time()
svc.fit(X_train, y_train)

err_test  = np.mean(y_test  != svc.predict(X_test))
print('Метод опорных векторов: ',err_test)
print('Время на обучение: ', time.time() - start_time, ' секунд')

Метод опорных векторов:  0.3567251461988304
Время на обучение:  0.019586563110351562  секунд


У Баггинга ошибка довольна близка к ошибки у лучших алгоритмов, при этом время его обучение значительно ниже. Этот алгоритм отлично подходит для решения задачи.

In [22]:
bc = BaggingClassifier()
start_time = time.time()
bc.fit(X_train, y_train)

err_test  = np.mean(y_test  != bc.predict(X_test))
print('Метод ближайших соседей: ',err_test)
print('Время на обучение: ', time.time() - start_time, ' секунд')

Метод ближайших соседей:  0.05263157894736842
Время на обучение:  0.05386948585510254  секунд


У Случайного леса ошибка минимальна и время близко лучшему при такой ошибки (Ада бустинг), соотвественно этот алгоритм тоже хорошо спровляется с задачей.

In [23]:
rfc = RandomForestClassifier()
start_time = time.time()
rfc.fit(X_train, y_train)

err_test  = np.mean(y_test  != rfc.predict(X_test))
print('Метод ближайших соседей: ',err_test)
print('Время на обучение: ', time.time() - start_time, ' секунд')

Метод ближайших соседей:  0.04678362573099415
Время на обучение:  0.19660520553588867  секунд


У Градиентного бустинга минимальная ошибка, но время почти в два раза хуже, чем у Случайного леса и почти в два с половиной раза хуже, чем у Ада бустинга при той же ошибки.

In [24]:
gbc = GradientBoostingClassifier()
start_time = time.time()
gbc.fit(X_train, y_train)

err_test  = np.mean(y_test  != gbc.predict(X_test))
print('Метод ближайших соседей: ',err_test)
print('Время на обучение: ', time.time() - start_time, ' секунд')

Метод ближайших соседей:  0.04093567251461988
Время на обучение:  0.40679383277893066  секунд


Ада бустинг показал наименьшие время работы среди алгоритмов с минимальной ошибкой. Вероятнее всего этот алгоритм наибольшим образом подходит для решения этой задачи.

In [25]:
abc = AdaBoostClassifier()
start_time = time.time()
abc.fit(X_train, y_train)

err_test  = np.mean(y_test  != abc.predict(X_test))
print('Метод ближайших соседей: ',err_test)
print('Время на обучение: ', time.time() - start_time, ' секунд')

Метод ближайших соседей:  0.04093567251461988
Время на обучение:  0.1495037078857422  секунд


В результате наиболее подходящими алгоритмами оказались Ада бустинг, Градиентный бустинг, Случайный лес, Баггинг, Деревья решений. 